# Сегментация изображений

In [ ]:
import os
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from PIL import Image

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F
from torch.autograd import set_detect_anomaly

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [ ]:
dst_path = '/content/drive/MyDrive/archive.zip'

In [ ]:
from zipfile import ZipFile
zipfile1 = ZipFile(dst_path)
zipfile1.extractall('/content/drive/MyDrive/archive')